# The Need for High-Order Methods

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.sparse.linalg as sla
import fd_tools as fd

In [ ]:
def first_deriv_matrix_per(n, dx, w):
    
    I, J, data = [], [], []
    p = (2*w+1)
    c = fd.fd_coeff(1, 0, dx*np.arange(-w, w+1))
    
    for i in range(n):
        I.extend(p*[i])
        J.extend([(i+k)%n for k in range(-w, w+1)])
        data.extend(c)
    
    return sp.csc_matrix((data, (I, J)))

In [ ]:
def matrix_demo():
    D = first_deriv_matrix_per(10, 1, 2)
    print(np.round(D.toarray(), 3))
    plt.spy(D)
    
matrix_demo()

In [ ]:
def periodic_grid(xl, xu, n):
    "Generates a periodic grid."

    dx = (xu-xl)/n
    return xl + np.arange(n)*dx

def periodic_advection_demo():
    
    # final time
    T = 20

    # spatial grid
    nx = 200
    L = 20
    x = periodic_grid(-L/2, L/2, nx)
    dx = x[1] - x[0]
    
    # initial pulse
    u0 = np.exp(-(x/4)**8)*np.sin(np.pi*x)
    
    for w in range(1,6):
        
        # create the derivative matrix
        D = first_deriv_matrix_per(nx, dx, w)

        # find the solution using a matrix exponential
        u = sla.expm(-D*T)*u0
        
        # plot the results
        plt.figure(figsize=(6,4))
        plt.plot(x, u0, '--')
        plt.plot(x, u)
        plt.title('w = {}'.format(w))
        
periodic_advection_demo()

In [ ]:
def dispersion_plots():

    # spatial grid
    nx = 200
    L = 20
    x = periodic_grid(-L/2, L/2, nx)
    dx = x[1] - x[0]
    
    plt.figure(figsize=(10,6))
    
    # modes to sample
    modes = 2*np.pi*np.arange(1,nx/2)/L

    # plot the exact dispersion curve
    plt.plot(modes, 0*modes+1, 'k--', label='exact')
    
    for w in range(5, 0, -1):
        
        # create the derivative matrix
        D = first_deriv_matrix_per(nx, dx, w)
        
        # initialize eigenvalue locations
        lam = np.zeros(len(modes))
        
        # loop over modes
        for i, xi in enumerate(modes):
            
            # approximate eigenvalues
            u = np.exp(1j*xi*x)
            Du = D*u
            lam[i] = np.linalg.norm(Du)/np.linalg.norm(u)
            
        # plot the numerical dispersion curve
        plt.plot(modes, lam/modes, label='w = {}'.format(w))
            
    plt.legend()
    
dispersion_plots()